# <center> **Analyse des trajets en Taxi à New York City**

**Ressources** :

lien des données : https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Description des variables des données sur les trajets en taxi jaune : https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

Comment importer un fichier PARQUET de données : https://www.nyc.gov/assets/tlc/downloads/pdf/working_parquet_format.pdf

Apache Arrow : https://arrow.apache.org/docs/python/parquet.html

## I/ Automatisation du téléchargement des données

![](new-york-1zwzte.jpg)

In [ ]:
#pip install pyarrow

In [1]:
import pyarrow.parquet as pq

In [2]:
df = pq.read_table("yellow_tripdata_2023-01.parquet")
type(df)

pyarrow.lib.Table

In [3]:
df = df.to_pandas()
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.shape

(3066766, 19)

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

### Téléchargement des données historiques

Commençons par télécharger tous les fichiers de données disponibles à ce jour.

In [6]:
import requests
import os

In [7]:
# Création d'un dossier pour stocker les fichiers téléchargés
os.makedirs("yellow_taxi_data", exist_ok=True)

In [12]:
for year in range(2023, 2010, -1):
    for month in range(1, 13):
        # Construction de l'url de téléchargement en fonction de l'année et du mois
        download_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
        file_name = f"yellow_taxi_data/yellow_tripdata_{year}-{month:02d}.parquet"
        
        # Télécharger le fichier
        print(f"Téléchargement de {file_name}...")
        response = requests.get(download_url)

        if response.status_code == 200:
            with open(file_name, "wb") as f:
                f.write(response.content)
        else:
            print(f"Impossible de télécharger {file_name}. Code de statut HTTP: {response.status_code}")

print("Téléchargement terminé!")

Téléchargement de yellow_taxi_data/yellow_tripdata_2023-01.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-02.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-03.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-04.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-05.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-06.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-07.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-08.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-09.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-10.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-11.parquet...
Téléchargement de yellow_taxi_data/yellow_tripdata_2023-12.parquet...
Impossible de télécharger yellow_taxi_data/yellow_tripdata_2023-12.parquet. Code de statut HTTP: 403
Téléchargement de yellow_taxi_data/yellow_tripdata_2022-01.

Comptons le nombre total de fichiers téléchargés

In [16]:
from os import listdir
from os.path import isfile, join
mypath = "./yellow_taxi_data"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['yellow_tripdata_2011-01.parquet',
 'yellow_tripdata_2011-02.parquet',
 'yellow_tripdata_2011-03.parquet',
 'yellow_tripdata_2011-04.parquet',
 'yellow_tripdata_2011-05.parquet',
 'yellow_tripdata_2011-06.parquet',
 'yellow_tripdata_2011-07.parquet',
 'yellow_tripdata_2011-08.parquet',
 'yellow_tripdata_2011-09.parquet',
 'yellow_tripdata_2011-10.parquet',
 'yellow_tripdata_2011-11.parquet',
 'yellow_tripdata_2011-12.parquet',
 'yellow_tripdata_2012-01.parquet',
 'yellow_tripdata_2012-02.parquet',
 'yellow_tripdata_2012-03.parquet',
 'yellow_tripdata_2012-04.parquet',
 'yellow_tripdata_2012-05.parquet',
 'yellow_tripdata_2012-06.parquet',
 'yellow_tripdata_2012-07.parquet',
 'yellow_tripdata_2012-08.parquet',
 'yellow_tripdata_2012-09.parquet',
 'yellow_tripdata_2012-10.parquet',
 'yellow_tripdata_2012-11.parquet',
 'yellow_tripdata_2012-12.parquet',
 'yellow_tripdata_2013-01.parquet',
 'yellow_tripdata_2013-02.parquet',
 'yellow_tripdata_2013-03.parquet',
 'yellow_tripdata_2013-04.pa

In [17]:
len(onlyfiles)

155

155 fichiers ont été téléchargé ce qui correpond exactement au nombre de fichiers de données des trajets en taxi jaune à New York disponibles à la date d'exécution du script (17/02/2024).

### Mise à jour du script de téléchargement des données 

 Les données sur les trajets seront publiées chaque mois, mais avec un retard de deux mois par rapport à la période couverte.  Par exemple, les données pour janvier 2024 ne seront publiées qu'en mars 2024.

  Pour prendre en compte le fait que les données seront publiées chaque mois à partir de 2023 et pour maintenir la flexibilité du script, nous pouvons utiliser la date actuelle pour déterminer la fourchette des années à parcourir. 
  
  Voici la version mise à jour du script :

In [18]:
from datetime import datetime, timedelta

In [24]:
# Fonction pour obtenir la date actuelle moins 2 mois
def get_previous_month():
    today = datetime.today()
    two_months_ago = today - timedelta(days=60)
    return two_months_ago.strftime("%Y-%m")

# Année courante
current_year = datetime.now().year
current_month = datetime.now().month

# Previous month
previous_month = get_previous_month()

# Mise à jour du script pour prendre en compte les données à venir
for year in range(2023, current_year + 1):
    for month in range(1, 13):
        # Ignorer les mois antérieurs à la date actuelle moins 2 mois
        if year == current_year and month > current_month:
            continue
        if f"{year}-{month:02d}" < previous_month:
            continue
            
        # Construction de l'url de téléchargement en fonction de l'année et du mois
        download_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
        file_name = f"yellow_taxi_data/yellow_tripdata_{year}-{month:02d}.parquet"
        
        # Télécharger le fichier
        print(f"Téléchargement de {file_name}...")
        response = requests.get(download_url)

        if response.status_code == 200:
            with open(file_name, "wb") as f:
                f.write(response.content)
        else:
            print(f"Impossible de télécharger {file_name}. Code de statut HTTP: {response.status_code}")

print("Téléchargement terminé!")

Téléchargement de yellow_taxi_data/yellow_tripdata_2023-12.parquet...
Impossible de télécharger yellow_taxi_data/yellow_tripdata_2023-12.parquet. Code de statut HTTP: 403
Téléchargement de yellow_taxi_data/yellow_tripdata_2024-01.parquet...
Impossible de télécharger yellow_taxi_data/yellow_tripdata_2024-01.parquet. Code de statut HTTP: 403
Téléchargement de yellow_taxi_data/yellow_tripdata_2024-02.parquet...
Impossible de télécharger yellow_taxi_data/yellow_tripdata_2024-02.parquet. Code de statut HTTP: 403
Téléchargement terminé!
